RRTMGP _g_-Point Reduction With Cost Function Optimization

## Dependencies

In [1]:
# standard modules
import os, sys, shutil, subprocess, time, multiprocessing

# pip installs
import netCDF4 as nc
import numpy as np

# local module paths
HOME = '/global/homes/p/pernak18'
GPTHOME = '{}/RRTMGP/g-point-reduction'.format(HOME)
PINCUSPATH = '{}/obsolete/pincus_k-distribution-opt'.format(GPTHOME)
LIBPATHS = [PINCUSPATH, 
            '{}/k-distribution-opt'.format(GPTHOME), 
            '{}/.local/'.format(HOME) + \
            'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages']
for path in LIBPATHS: sys.path.append(path)

# local modules
from combine_gpoints_fn import combine_gpoints_fn
#from prepare_cpp_input import prepare_input
import cost_function as CF
import ref_values as RV

## Function Definitions

In [42]:
def pathCheck(path):
    """
    Determine if file exists. If not, throw an Assertion Exception
    """

    assert os.path.exists(path), 'Could not find {}'.format(path)
    
def link_rrtmgp(rrtmgpPath="{}/rte-rrtmgp-cpp".format(GPTHOME), 
                exe='./test_garand', 
                coeffSW='rrtmgp-data-sw-g224-2018-12-04.nc', 
                coeffLW='rrtmgp-data-lw-g256-2018-12-04.nc', 
                cloudSW='rrtmgp-cloud-optics-coeffs-sw.nc', 
                cloudLW='rrtmgp-cloud-optics-coeffs-lw.nc'):

    """
    File staging for RRTMGP
    
    Call
        
    Inputs
        fThermal -- int, optimize LW k-distribution
    
    Outputs
        None

    Keywords
        rrtmgpPath -- string, top-level directory with RRTMGP code
        exe -- string, 
            absolete path to RRTMGP LW/SW solver (flux calulation) executable
        coeffSW -- string, 
            basename of file with SW absorption coefficients for clear skies
        coeffLW -- string, 
            basename of file with LW absorption coefficients for clear skies
        cloudSW -- string, 
            basename of file with SW absorption coefficients with clouds
        cloudLW -- string,
            basename of file with LW absorption coefficients with clouds
    """
    
    #buildDir = '{}/build'.format(rrtmgpPath)
    coeffDir = '{}/rte-rrtmgp/rrtmgp/data'.format(rrtmgpPath)
    cloudDir = '{}/rte-rrtmgp/extensions/cloud_optics'.format(rrtmgpPath)

    paths = [exe, coeffDir, cloudDir, 
             '{}/{}'.format(coeffDir, coeffSW), 
             '{}/{}'.format(coeffDir, coeffLW), 
             '{}/{}'.format(cloudDir, cloudSW), 
             '{}/{}'.format(cloudDir, cloudLW)]
    for path in paths: pathCheck(path)

    exeBase = os.path.basename(exe)
    if not os.path.exists(exeBase): os.symlink(exe, exeBase)

    swTarget = 'coefficients_sw.nc'
    # Pernak: reverse the logic here? done
    if not os.path.exists(swTarget) and not THERMAL:
        os.symlink('{}/{}'.format(coeffDir, coeffSW), swTarget)

    lwTarget = 'coefficients_lw.nc'
    # Pernak: reverse the logic here? done
    if not os.path.exists(lwTarget) and THERMAL: 
        os.symlink('{}/{}'.format(coeffDir, coeffLW), lwTarget)

    swTarget = 'cloud_coefficients_sw.nc'
    if not os.path.exists(swTarget):
        os.symlink('{}/{}'.format(cloudDir, cloudSW), swTarget)

    lwTarget = 'cloud_coefficients_lw.nc'
    if not os.path.exists(lwTarget):
        os.symlink('{}/{}'.format(cloudDir, cloudLW), lwTarget)

def process_costs(components):
    totCost = CF.total_cost(components)
    print("Total cost: {}".format(totCost))
    return np.append(components, totCost)

def append_errors(idx, errorDict, errWin, errOld):
    data = [errorDict[idx][0] + errWin[0] - errOld[0], 
            errorDict[idx][1] + errWin[1] - errOld[1]]
    errorDict[idx] = list(data)
    return([0]) 

def modify_wt_gpt(gOut, wt, bandID, nGpt, gptArr):
    wt[gOut-1] = wt[gOut-1]+wt[gOut]
    wt = np.delete(wt, gOut)
    bandID = np.delete(bandID, gOut)
    return wt, bandID, nGpt-1, gptArr.copy()

def remove_temp_files(dirRes):
    for d in ['{}/coeffs'.format(dirRes), '{}/fluxes'.format(dirRes)]:
        for f in os.listdir(d): 
            if not f.startswith("temp"):
                os.remove(os.path.join(d, f))

def write_cost_nc(dirOut, allBands, allGpts, allCosts, names, 
                  icost=2, f_thermal=False, f_errtype=5):

    ncFile = "{}/optimisation_output.{}.cost{:02d}.norm{:01d}.bnd{:02d}.nc".format(
        dirOut, 'lw' if f_thermal else 'sw', icost, f_errtype, bands[0])
    os.makedirs(os.path.dirname(ncFile), exist_ok=True)
    dataOut = nc.Dataset(ncFile, 'w')
    
    dataOut.createDimension("n_name", len(names))
    dataOut.createDimension("pair", 2)
    dataOut.createDimension("n_iter", allGpts.shape[1])
    dataOut.createDimension("t_iter", allGpts.shape[1]+1)

    gptsOut  = data_out.createVariable(
        "combined_gpt_pair", "f4", ("pair", "n_iter"))
    bandOut  = data_out.createVariable(
        "combined_band", "f4", ("n_iter"))
    costOut  = data_out.createVariable(
        "cost_components", "f4", ("n_name", "t_iter"))
    ctotOut  = data_out.createVariable(
        "total_cost", "f4", ("t_iter"))
    nameOut  = data_out.createVariable(
        "names", str, ("n_name"))
    
    gptsOut[:] = all_gpts
    bandOut[:] = all_bands
    costOut[:] = all_costs[:-1]
    ctotOut[:] = all_costs[-1]
    for i in range(len(names)): nameOut[i] = names[i]
    
    dataOut.close()

def make_name_variant(iterNum, gpts):
    # Return string of form iterNNN.GGG.GGG,
    #   iterNum is a scalar int, gpts is list of ints of length 2
    return('iter{0:03d}.{1[0]:03d}.{1[1]:03d}'.format(iterNum, gpts))

def trial_cost_function(iterNum, dirRes, dirData, 
                        start_kdist_file, gpts, wts, 
                        ref_flux_file, cf_norm, flux_data, 
                        f_thermal=False, f_forcing=False, 
                        exe='./rrtmgp_garand_atmos'):

    # Return the terms in the cost function for a proposed 
    # combination of two gpoints from an initial set
    trial_kdist_file = '{}/coeffs/coefficients_{}.nc'.format(
        dirRes, make_name_variant(iterNum, gpts))
    trial_flux_file  = '{}/fluxes/fluxes.all.{}.nc'.format(
        dirRes, make_name_variant(iterNum, gpts))
    
    # Make sure directories exist
    os.makedirs(os.path.dirname(trial_kdist_file), exist_ok=True)
    os.makedirs(os.path.dirname(trial_flux_file), exist_ok=True)

    # Combine the pair of g-points specified in gpts(2) with weights wts(2)
    combine_gpoints_fn(start_kdist_file, trial_kdist_file, gpts, wts)
    
    # create and go to temporary dir
    output_path = '{}/fluxes/temp_{}/'.format(
        dirRes, make_name_variant(iterNum, gpts))
    os.makedirs(output_path, exist_ok=True)
    os.chdir(output_path)

    # stage trial coefficient files
    domain = 'lw' if f_thermal else 'sw'
    link_rrtmgp(exe=EXE)
    file_rrtmgp_output = "rte_rrtmgp_output_{}.nc".format(domain)
    file_rrtmgp_input = "{}/input/rte_rrtmgp_input_{}.nc".format(
        dirData, domain)
    coeffStage = "coefficients_{}.nc".format(domain)
    shutil.copy2(trial_kdist_file, coeffStage)

    # link input files
    for icase in FRANGE: shutil.copy2(file_rrtmgp_input.format(icase), './')
    
    # run and combine fluxes
    subprocess.run([exe, file_rrtmgp_input, coeffStage])
    os.rename(file_rrtmgp_input, trial_flux_file)
    """
    subprocess.run(["./test_garand", 
                    "--no-{}".format('shortwave' if f_thermal else 'longwave'), 
                    "--output-bnd-fluxes"] + \
                   (['--forcing'] if f_forcing else []))

    subprocess.run(['/global/common/sw/cray/cnl7/haswell/nco/' + \
                    '4.7.9/gcc/8.3.0/bz3muff/bin/ncecat'] + \
                   [file_rrtmgp_output.format(i) for i in fRange] + \
                   ["-u", "record", "-O", trial_flux_file])
    """

    os.chdir(kDistOptPath)
    shutil.rmtree(output_path)
    
    # Return the cost function components normalized by error at iteration 0
    flux_data[gpts[0]] = CF.cost_function_components(trial_flux_file, ref_flux_file)
    return([0])

## Cost Function Selection

Currently, there are 8 different cost functions that can be minimized. The default is 5, but other options for `icost` (type `int`) are:
  1. 
  2. 
  3. 
  4. 
  5. 
  6. 
  7. 
  8. 

In [43]:
# Global variable definition of the cost function type
ICOST = 5

if ICOST == 1: 
    names = ['dn_bnd_lwr', 'up_bnd_lwr', 'nt_bnd_lw', 'dn_tot_lwr', 'up_tot_lwr', 'nt_tot_lwr', 'heat_lwr', 
             'dn_bnd_upr', 'up_bnd_upr', 'nt_bnd_up', 'dn_tot_upr', 'up_tot_upr', 'nt_tot_upr', 'heat_upr']
elif ICOST == 2:
    names = ['nt_bnd_lw', 'nt_tot_lwr', 'heat_lwr', 'nt_bnd_up', 'nt_tot_upr', 'heat_upr']
elif ICOST == 3:
    names = ['nt_bnd_lwr', 'nt_tot_lwr', 'heat_lwr']
elif ICOST == 4:
    names = ['nt_bnd_upr', 'nt_tot_upr', 'heat_upr']
elif ICOST == 5:
    names = ['sfc_flux']
elif ICOST == 6:
    names = ['heat_lwr', 'heat_upr', 'sfc_flux']
elif ICOST == 7:
    names = ['sfc_flux_dn', 'trp_flx_dn', 'trp_flx_up', 'toa_flx_up']
elif ICOST == 8:
    names = ['heat_thermo']
else:
    print('INVALID COST FUNCTION')
    
print('Variables in cost function: {}'.format(', '.join(names)))

Variables in cost function: sfc_flux


## Additional Global Variables

Now define what kind of normalization `f_errtype` (type `int`) will be applied to the cost function:

  0. absolute error (RMSE)
  1. change in RMSE with respect to RMSE at iteration 0
  2. error w.r.t. LBLRTM (normalized rmse)

Whether the _k_-distribution optimization is done in the longwave or shortwave domain is specified with `f_thermal` (type `bool`). This is important for filename specifications. Forcing has not been implemented in this notebook, so it should **always** be set to `False`.

In [44]:
ERRTYPE = 2
THERMAL = True
FORCING = False
FRANGE = range(1+6*(THERMAL and FORCING))
DOMAIN = 'lw' if THERMAL else 'sw'
DOMAINL = 'longwave' if THERMAL else 'shortwave'
EXE = '{}/rrtmgp_garand_atmos'.format(PINCUSPATH)

Note that the executable is the [FORTRAN-based](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/rrtmgp_garand_atmos.F90) `rrtmgp_garand_atmos` rather than the [C++-based](https://github.com/MennoVeerman/k-distribution-opt/blob/master/test_garand.cpp) `test_garand`. The latter was yielding `Segmentation Faults` at runtime on the NERSC `cori` machine and thus was not producing the results that were needed for the rest of the optimization. Because of these errors, we pursue the "Hillman" method in the rest of this notebook, while attempting to fold in the enhancements from Menno, which include cost function flexibility and diagnostics plotting.

## Path Assignments

In [45]:
kDistOptPath = LIBPATHS[0] # path of k-distribution directory (originally `bpath`)
pathCheck(kDistOptPath)

# where to save coeff and temporary flux files (originally `dpath`)
outDatPath = '{}/intermediate_files'.format(GPTHOME)
if not os.path.isdir(outDatPath): os.makedirs(outDatPath)

#file_LBLRTM = '{}/lbl_reference_{}.nc'.format(kDistOptPath, DOMAINL)
# newest reference file provided by Robert (with `record` dimension)
projectDir = '/project/projectdirs/e3sm/pernak18/inputs/g-point-reduce'
file_LBLRTM = '{}/lblrtm-{}-flux-inputs-outputs-garandANDpreind.nc'.format(
    projectDir, DOMAIN)

# RRTMGP coefficient file
file_coeffORG = 'rrtmgp-data-lw-g256-2018-12-04.nc' if THERMAL else \
    'rrtmgp-data-sw-g224-2018-12-04.nc'
file_coeffORG_dir = '{}/obsolete/rte-rrtmgp/rrtmgp/data/'.format(
    GPTHOME)
pathCheck(file_coeffORG_dir)

# Directory where new fluxes and coefficient files are stored.
dirOut  = '{}/outputs_bnd2'.format(outDatPath)
dirRes  = "{}/results_bnd2/{}.cost{:02d}.norm{:01d}/".format(
    outDatPath, DOMAIN, ICOST, ERRTYPE)
dirData = "{}/data_bnd2/{}.cost{:02d}.norm{:01d}/".format(
    outDatPath, DOMAIN, ICOST, ERRTYPE)
PATHS = [dirOut, dirRes, dirData]
for path in PATHS:
    if not os.path.isdir(path): os.makedirs(path)

## main()

### Runtime Variables Definition, Extract Information from LBLRTM Reference Results

We will use the pressures that were input into LBLRTM. `p_lev` is a (1 x 43 x 42) array, so we are extracting the entire pressure profile of the first Garand atmosphere (in descending order, so surface-to-TOA).

The number of _g_-points can be extracted from the LBL netCDF as well. The weights associated with each _g_-point are the same for each band, so we effectively produce an `nGpt`x`nBnds` array, then flatten it to a 1-D vector.

In [46]:
# pressures (1D)
p_lev = nc.Dataset(file_LBLRTM).variables['p_lev'][0,:,0]

# Initial settings.
# Todo: generalize, read values from coefficient file
# Number of bands
nBnds = len(nc.Dataset(file_LBLRTM).variables['band'][:])

# Number of G-points in each band.
nGptsPerBandOrg = 16

# Number of G-points
nGpt = nBnds * nGptsPerBandOrg

# optimization iterations
nOptIt = 210

# Band ID for each G-point
bandID = range(1, nBnds+1)
bandID = np.repeat(bandID, nGptsPerBandOrg)

# G-point weights (same for all bands)
# expand weights for one band to the rest of the bands with np.tile
# so weights are an (nGpt x nBnds)-element vector
# a 
wtORG = [0.1527534276, 0.1491729617, 0.1420961469, 0.1316886544, 
         0.1181945205, 0.1019300893, 0.0832767040, 0.0626720116, 
         0.0424925000, 0.0046269894, 0.0038279891, 0.0030260086, 
         0.0022199750, 0.0014140010, 0.0005330000, 0.0000750000]
wt = np.tile(wtORG, nBnds)

### RRTMGP File Staging

In [47]:
# Prepare input file
file_rrtmgp_input = "{}/input/rte_rrtmgp_input_{}.nc".format(
    dirData, DOMAIN)
os.makedirs(os.path.dirname(file_rrtmgp_input), exist_ok=True)
#prepare_input(file_LBLRTM, file_rrtmgp_input)

# Create output directory
file_rrtmgp_output = "{}/fluxes/rte_rrtmgp_output_{}.nc".format(
    dirData, DOMAIN)
os.makedirs(os.path.dirname(file_rrtmgp_output), exist_ok=True)

# copy original k-dist file
shutil.copy2('{}/{}'.format(file_coeffORG_dir, file_coeffORG), 
             '{}/{}'.format(dirData, file_coeffORG))

'/global/homes/p/pernak18/RRTMGP/g-point-reduction/intermediate_files/data_bnd2/lw.cost05.norm2//rrtmgp-data-lw-g256-2018-12-04.nc'

### Initial RRTMGP Fluxes (With Original _k_-distribution)/Reference Fluxes

Stage some more files into a working directory (where the model is run over many iterations), then perform an initial run of RRTMGP over all Garand atmospheres. `returncode` of 0 means success. Be leery of other return codes.

In [48]:
os.chdir(dirRes)

# copy k-distribution and profile information files to working dir
# we are using Robert's LBL reference file for the latter
# https://github.com/RobertPincus/k-distribution-opt/blob/master/lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc
coeffNC = 'coefficients_{}.nc'.format(DOMAIN)
shutil.copy2('{}/{}'.format(dirData, file_coeffORG), coeffNC)
inNC = 'rte_rrtmgp_input-output_0.nc'
shutil.copy2(file_LBLRTM, './{}'.format(inNC))

# run Robert's version of `test_garand`
subprocess.run([EXE, inNC, coeffNC])

CompletedProcess(args=['/global/homes/p/pernak18/RRTMGP/g-point-reduction/obsolete/pincus_k-distribution-opt/rrtmgp_garand_atmos', 'rte_rrtmgp_input-output_0.nc', 'coefficients_lw.nc'], returncode=0)

In [49]:
if not os.path.isdir('{}/fluxes'.format(dirRes)):
    os.makedirs('{}/fluxes'.format(dirRes))

file_rrtmgp_ref = dirRes + "fluxes/reference_kdist_fluxes.nc"
os.rename(inNC, file_rrtmgp_ref)

# location of `ncecat` NCO (netCDF Operators) executable on 
# NERSC machines (module load nco)
# Robert's LBLRTM reference file already has a `record` dimension, 
# so this `ncecat` does not need to be done
# subprocess.run(['/global/common/sw/cray/cnl7/haswell/nco/' + \
#                 '4.7.9/gcc/8.3.0/bz3muff/bin/ncecat'] + \
#                ['rte_rrtmgp_output_{}.nc'.format(i) for i in FRANGE] + \
#                ['-u', 'record', '-O', file_rrtmgp_ref])


In [50]:
os.chdir(kDistOptPath)
# think about a way to incorporate Robert's 
# https://github.com/RobertPincus/k-distribution-opt/blob/master/cost_function_2.py
"""
if ERRTYPE == 1:
    cf_norm = CF.cost_function_from_error(
        CF.flux_heat_errors(file_rrtmgp_ref, file_LBLRTM), p_lev, ICOST)
    cf_norm_norm = CF.normalized_costs(cf_norm, cf_norm, ERRTYPE)
elif ERRTYPE == 2 or ERRTYPE == 0:
    cf_norm = RV.reference_values(
        RV.get_fluxes(file_LBLRTM), p_lev, ICOST)
    cf_frst = CF.cost_function_from_error(
        CF.flux_heat_errors(file_rrtmgp_ref, file_LBLRTM), p_lev, ICOST)
    cf_norm_norm = CF.normalized_costs(cf_frst, cf_norm, ERRTYPE)
winner_err_old = CF.flux_heat_errors(file_rrtmgp_ref, file_LBLRTM)
"""
cf_norm = CF.cost_function_components(file_rrtmgp_ref, file_LBLRTM)

### Array Initialization for Greedy Optimization of Cost Function

Need to make the calculations more transparent for Eli.

## Hillman Version

In [51]:
os.makedirs('data', exist_ok=True)

# On first iteration, coefficients file is the original one
shutil.copy2('{}/{}'.format(file_coeffORG_dir, file_coeffORG), 
             'data/{}'.format(file_coeffORG))
file_coeffFromPreviousIteration = str(file_coeffORG)

print(f'Combining g-point pairs {nOptIt} times...'); sys.stdout.flush()

error_dict = multiprocessing.Manager().dict()
for iMain in range(1,nOptIt):
    # Which coefficient file to use? 
    file_coeff = 'data/'+file_coeffFromPreviousIteration
    pathCheck(file_coeff)

    # Create list of all adjacent g-point pairs in each band
    gpt_list = [[x,x+1] for x in range(1,nGpt) if bandID[x-1] == bandID[x]]
    wgt_list = [(wt[gpt_pair[0]-1],wt[gpt_pair[1]-1]) for gpt_pair in gpt_list]

    # Compute error terms for each combination of adjacent g-points pairs
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        # separate processes, each with their own arguments
        res = [pool.apply_async(trial_cost_function, 
           args = (iMain, dirRes, dirData, file_coeff, 
                  gpt_pair, wgt_pair, file_LBLRTM, 
                  cf_norm, error_dict, THERMAL, FORCING, exe=EXE)) \
           for gpt_pair, wgt_pair in zip(gpt_list, wgt_list)]
        temp  = [r.get() for r in res]
        #results = [pool.apply_async(trial_cost_function, 
        #                            args=(iMain, file_coeff, gpt_pair, wgt_pair, file_LBLRTM, cf_norm))
        #            for gpt_pair, wgt_pair in zip(gpt_list, wgt_list)]
        #cfn_list = [r.get() for r in results]
        break

    ##########################################################################################
    #
    # Greedy optimization
    # Of all the g-point combinations in this iteration, which had the smallest error?
    #
    winner = np.argmin([cf.total_cost(x) for x in cfn_list])
    #
    # Set the new coefficent file for the next iteration.
    #
    file_coeffFromPreviousIteration = 'coefficients_{0}.nc'.format(make_name_variant(iMain, gpt_list[winner]))
    shutil.copy2('results/coeffs/'+file_coeffFromPreviousIteration, 'data/'+file_coeffFromPreviousIteration)
    #
    # Remove temporary files
    #
    for d in ['results/coeffs', 'results/fluxes']:
        for f in os.listdir(d): os.remove(os.path.join(d, f))

    g1out = int(gpt_list[winner][0])
    g2out = int(gpt_list[winner][1])
    print("For iteration {0:03d},combining g-points {1[0]:03d} and {1[1]:03d} in band {2:02d}".format(iMain, gpt_list[winner], bandID[g1out-1]))
    print("   F_:  ",cfn_list[winner][0].values)
    print("   H_:  ",cfn_list[winner][1].values)
    print("   FO_: ",cfn_list[winner][2].values)
    print("   S_:  ",cfn_list[winner][3].values)
    print("   tot: ",cf.total_cost(cfn_list[winner]))
    print("   New coefficient file: ",file_coeffFromPreviousIteration)

    # For the next iteration, modify the weights and the array that contains the
    # number of G-points in each band.
    # Next iteration will be over nGpts-1
    wt[g1out-1] = wt[g1out-1]+wt[g2out-1]
    wt     = np.delete(wt,     g2out-1)
    bandID = np.delete(bandID, g2out-1)
    nGpt = nGpt-1
    ##########################################################################################
    #
    # Save terms of the cost function for every possible g-point combination
    #
    dataOUT = netCDF4.Dataset("data/cost_function_terms.lw.iter"+str(iMain).zfill(3)+".nc",'w')
    dataOUT.createDimension("Gpt_combination",2)
    dataOUT.createDimension("Case",len(gpt_list))
    H_norm  = dataOUT.createVariable("H_norm",        "f4", ("Case"))
    F_norm  = dataOUT.createVariable("F_norm",        "f4", ("Case"))
    FO_norm = dataOUT.createVariable("FO_norm",       "f4", ("Case"))
    S_norm  = dataOUT.createVariable("S_norm",        "f4", ("Case"))
    gPtsOut = dataOUT.createVariable("Gpt_pair",      "f4", ("Case","Gpt_combination"))
    cstfnc  = dataOUT.createVariable("cost_function", "f4", ("Case"))

    for i in np.arange(len(gpt_list)):
        # Store data
        F_norm [i]  = cfn_list[i][0]
        H_norm [i]  = cfn_list[i][1]
        FO_norm[i]  = cfn_list[i][2]
        S_norm [i]  = cfn_list[i][3]
        gPtsOut[i,:] = gpt_list[i]
        cstfnc[i]    = cf.total_cost(cfn_list[i])

    # Close output file
    dataOUT.close()

SyntaxError: invalid syntax (<ipython-input-51-7c022bc00483>, line 26)

#### Menno version

[Menno's Optimizer](https://github.com/MennoVeerman/k-distribution-opt/blob/master/optimizer.py) and Ben Hillman's (Ben separates into [LW](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/optimizeLW.py) and [SW](https://github.com/RobertPincus/k-distribution-opt/blob/brhillman/dev/optimizeSW.py) optimizers) are pretty similar. Menno experiments with many more cost functions and plots intermediate results more often, but there seems to be some inconsistencies between the input files and perhaps executables that are used. For example, Menno uses `ncecat` to add a `record` dimension to the netCDF files, but Robert's and Ben's `rrtmgp_garand_atmos` expects it at runtime. Since we used `rrtmgp_garand_atmos`, we will use Ben's approach to optimization.

In [29]:
# Store cost of reference (original k-distribution)
all_costs = np.zeros((len(names)+1, nOptIt+1))
all_bands = np.zeros((nOptIt))
all_gpts  = np.zeros((2, nOptIt))
for c in cf_norm: print(c.shape)
sys.exit()

print('For iteration 000, combined nothing yet')
all_costs[:, 0] = process_costs(cf_norm)

(3,)
(42,)
(6, 3)
(3, 16)


SystemExit: 

In [ ]:
print(f'Combining g-point pairs {nOptIt} times...'); sys.stdout.flush()

############################################
# First iteration: outside main loop
############################################
gpt_list = [[x,x+1] for x in range(1, nGpt) if bandID[x-1] == bandID[x]][:]
wgt_list = [(wt[gpt_pair[0]-1], wt[gpt_pair[1]-1]) for gpt_pair in gpt_list][:]
gpt_arr = np.array(gpt_list)[:,0]

file_coeffFromPreviousIteration = file_coeffORG
file_coeff = dirData + file_coeffFromPreviousIteration

# Compute and store error terms for each combination of adjacent g-points pairs
# call `trial_cost_function` multiple times in parallel ("pool" the processes)
error_dict = multiprocessing.Manager().dict()
with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    # separate processes, each with their own arguments
    res = [pool.apply_async(trial_cost_function, 
           args = (1, dirRes, dirData, file_coeff, 
                  gpt_pair, wgt_pair, file_LBLRTM, 
                  cf_norm, error_dict, THERMAL, FORCING)) \
           for gpt_pair, wgt_pair in zip(gpt_list, wgt_list)]
    temp  = [r.get() for r in res]

# Greedy optimization: which g-point combination has the smallest error?
cost_list = [cf.normalized_costs(cf.cost_function_from_error(
    error_dict[i], p_lev, ICOST), cf_norm, ERRTYPE) for i in gpt_arr]
winner_idx = np.argmin([cf.total_cost(i) for i in cost_list])
winner_err = error_dict[gpt_list[winner_idx][0]]

# Set the new coefficent file for the next iteration.
file_coeffFromPreviousIteration = 'coefficients_{0}.nc'.format(
    make_name_variant(1, gpt_list[winner_idx]))
shutil.copy2('{}/coeffs/{}'.format(dirRes, file_coeffFromPreviousIteration), 
             '{}/{}'.format(dirData, file_coeffFromPreviousIteration))

# Remove temporary files
remove_temp_files(dirRes)

#process costs
g_out = int(gpt_list[winner_idx][0])
all_bands[0]   = bandID[g_out]
all_gpts [:,0] = gpt_list[winner_idx]
all_costs[:,1] = process_costs(cost_list[winner_idx])

# Modify the weights the g-point arrays
wt, bandID, nGpt, gpt_arr_old = modify_wt_gpt(g_out, wt, bandID, nGpt, gpt_arr)
print('For iteration 001, ', end='')
print('combining g-points {0[0]:03d} and {0[1]:03d} in band {1:02d}'.format(
    gpt_list[winner_idx], bandID[g_out-1]))
print("New coefficient file: ", file_coeffFromPreviousIteration)

# now perform the rest of the iterations
for iiter in range(nOptIt-1):
    iMain = iiter + 2

    # Which coefficient file to use? On first iteration, this is original 
    # coefficient file
    file_coeff = '{}/{}'.format(dirData, file_coeffFromPreviousIteration)
    
    # Create list of all adjacent g-point pairs in each band
    gpt_list = [[x,x+1] for x in range(1,nGpt) if \
                bandID[x-1] == bandID[x]][:]
    wgt_list = [(wt[gpt_pair[0]-1],wt[gpt_pair[1]-1]) for \
                gpt_pair in gpt_list][:]
    gpt_arr = np.array(gpt_list)[:,0]
    gpt_win = gpt_arr[winner_idx-1:winner_idx+1]
    los_arr = np.delete(
        gpt_arr, [winner_idx-1] + [winner_idx] * (winner_idx<len(gpt_arr)))

    #rearrange error dict
    error_dict.pop(g_out)
    for igpt in range(winner_idx,len(gpt_arr)):
        error_dict[gpt_arr[igpt]] = error_dict.pop(gpt_arr_old[igpt+1])
    
    ### Compute error terms for each combination of adjacent g-points pairs
    # recompute with rte+rrtmgp
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        res = [pool.apply_async(trial_cost_function, 
               args = (iMain, dirRes, dirData, file_coeff, 
                      gpt_pair, wgt_pair, file_LBLRTM, 
                      cf_norm, error_dict, THERMAL, FORCING)) \
               for gpt_pair, wgt_pair in zip(
                      gpt_list[winner_idx-1:winner_idx+1], 
                      wgt_list[winner_idx-1:winner_idx+1])]
        temp  = [r.get() for r in res]

    # Add errors from previous iteration
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        res = [pool.apply_async(append_errors, args = \
              (idx, error_dict, winner_err, winner_err_old)) for idx in los_arr]
        temp  = [r.get() for r in res]
    cost_list = [cf.normalized_costs(cf.cost_function_from_error(
        error_dict[i], p_lev, ICOST), cf_norm, ERRTYPE) for i in gpt_arr]
    
    # Greedy optimization: which g-point combination has the smallest error?
    winner_idx = np.argmin([cf.total_cost(i) for i in cost_list])

    # New reference errors
    winner_err_old = winner_err
    winner_err = error_dict[gpt_list[winner_idx][0]]
    
    # Set the new coefficent file for the next iteration.
    file_coeff_new = 'coefficients_{0}.nc'.format(
        make_name_variant(iMain, gpt_list[winner_idx]))
    if gpt_list[winner_idx][0] not in gpt_win: 
        combine_gpoints_fn(file_coeff, '{}/coeffs/{}'.format(
            dirRes, file_coeff_new, gpt_list[winner_idx], wgt_list[winner_idx])
        
    file_coeffFromPreviousIteration = file_coeff_new
    shutil.copy2('{}/coeffs/{}'.format(dirRes, file_coeffFromPreviousIteration), 
                 '{}/{}'.format(dirData, file_coeffFromPreviousIteration))
    
    # Remove temporary files
    remove_temp_files(dirRes)

    # Process costs
    g_out = int(gpt_list[winner_idx][0])
    all_bands[iMain-1] = bandID[g_out]
    all_gpts [:,iMain-1] = gpt_list[winner_idx]
    all_costs[:,iMain] = process_costs(cost_list[winner_idx])
    
    # Modify the weights the g-point arrays
    wt, bandID, nGpt, gpt_arr_old = \
        modify_wt_gpt(g_out, wt, bandID, nGpt, gpt_arr)
    print('For iteration {0:03d}, '.format(iMain), end='')
    print('combining g-points {0[0]:03d} and {0[1]:03d} in band {1:02d}'.format(
        iMain, gpt_list[winner_idx], bandID[g_out-1]))
    print('New coefficient file: {}'.format(file_coeffFromPreviousIteration))
    
write_cost_nc(all_bands, all_gpts, all_costs, names, 
             icost=ICOST, f_thermal=THERMAL, f_errtype=ERRTYPE)